In [100]:
import sys
import os
sys.path.insert(0, os.path.abspath(os.path.join(".", "src")))

from micrograd.engine import Value

In [136]:
import random
import typing

class Neuron:
    def __init__(self, nin: int) -> None:
        self.w = [Value(random.uniform(-1, 1)) for _ in range(nin)]
        self.b = Value(random.uniform(-1, 1))

    def __call__(self, x: list[float] | list[Value]) -> Value:
        # input may be wrapped in Value already, or cast here
        _x = x if isinstance(x[0], Value) else [Value(v) for v in x]
        # w * x + b
        activation = sum((wi * xi for wi, xi in zip(self.w, _x)), self.b)
        return activation.tanh()
    
    def parameters(self) -> list[Value]:
        return self.w + [self.b]
    
class Layer:
    def __init__(self, nin: int, nout: int) -> None:
        self.neurons = [Neuron(nin) for _ in range(nout)]

    def __call__(self, x: list[float] | list[Value]) -> list[Value] | Value:
        outs = [n(x) for n in self.neurons]
        return outs[0] if len(outs) == 1 else outs
    
    def parameters(self) -> list[Value]:
        return [p for neuron in self.neurons for p in neuron.parameters()]

class MLP:
    def __init__(self, nin: int, nouts: list[int]) -> None:
        sz = [nin] + nouts
        self.layers = [Layer(sz[i], sz[i+1]) for i in range(len(nouts))]

    def __call__(self, x: list[float] | list[Value]) -> Value:
        _x = x if isinstance(x[0], Value) else [Value(v) for v in x]
        _x = typing.cast(list[Value], _x)

        for layer in self.layers:
            r = layer(_x)
            _x = [r] if isinstance(r, Value) else r

        assert len(_x) == 1, "broken invariant"
        return _x[0]
    
    def parameters(self) -> list[Value]:
        return [p for layer in self.layers for p in layer.parameters()]

In [151]:
n = MLP(3, [4, 4, 1])

In [152]:
xs = [
    [2.0, 3.0, -1.0],
    [3.0, -1.0, 0.5],
    [0.5, 1.0, 1.0],
    [1.0, 1.0, -1.0]
]
ys = [1.0, -1.0, -1.0, 1.0]

In [153]:
learning_rate = 0.05
for k in range(32):
    # forward pass
    ypred = [n(x) for x in xs]
    loss = sum([(yout - ygt)**2 for ygt, yout in zip(ys, ypred)])
    assert isinstance(loss, Value)

    # zero grad
    for p in n.parameters():
        p.grad = 0.0

    # backward pass
    loss.backward()

    # parameter update
    for p in n.parameters():
        p.data += -learning_rate * p.grad

    print(f"step = {k}, loss = {loss.data:.4f}")

step = 0, loss = 5.3615
step = 1, loss = 3.5068
step = 2, loss = 2.4456
step = 3, loss = 1.7492
step = 4, loss = 1.1064
step = 5, loss = 0.6460
step = 6, loss = 0.3984
step = 7, loss = 0.2724
step = 8, loss = 0.2020
step = 9, loss = 0.1586
step = 10, loss = 0.1295
step = 11, loss = 0.1090
step = 12, loss = 0.0937
step = 13, loss = 0.0820
step = 14, loss = 0.0728
step = 15, loss = 0.0653
step = 16, loss = 0.0592
step = 17, loss = 0.0541
step = 18, loss = 0.0497
step = 19, loss = 0.0460
step = 20, loss = 0.0427
step = 21, loss = 0.0399
step = 22, loss = 0.0374
step = 23, loss = 0.0352
step = 24, loss = 0.0332
step = 25, loss = 0.0315
step = 26, loss = 0.0299
step = 27, loss = 0.0284
step = 28, loss = 0.0271
step = 29, loss = 0.0259
step = 30, loss = 0.0248
step = 31, loss = 0.0237


In [154]:
ypred

[Value(data=0.9690009811618984),
 Value(data=-0.9087352708144134),
 Value(data=-0.9274951960955903),
 Value(data=0.9041002208974136)]